<a href="https://colab.research.google.com/github/fredericokrohling/1-Python-MBA/blob/main/Video%2020%20Pandas%20Prepara%C3%A7%C3%A3o_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**negrito**PREPARACAO DE DADOS
1. Cada variavel forma uma coluna
2. Cada observacao forma uma linha
3. Cada tipo de unidade observacional forma uma tabela

# 3 forma normal de Codd!

In [14]:
# Recebemos 2 tabelas, cada uma com temperaturas coletadas em cidades.
#A primeira é referente aos primeiros dias de coleta
# a segunda aos dias seguintes, organizadas por outra pessoa e enviada
# Precisamos preparar os dados de forma que uma observacao seja:
# 1. uma cidade, um dia e uma temperatura naquele dia

# Criando os DF para o exemplo
import numpy as np
import pandas as pd

temps = np.array([[23, 21, 20],[30, 29, 28],[18, 21, 20],[9, 10, 13]])

dtemp1 = pd.DataFrame(temps, index=['São Paulo','Fortaleza','Montevideo','London'])

dtemp2 = pd.DataFrame({
    'São Paulo': [25, 27, 23, 25],
    'Fortaleza': [35, 32, 31, 29],
    'Montevideo': [20, 18, 21, 23],
    'London': [14, 15, 12, 13]
})

# O problema acima é que cada tabela foi formatada de uma maneira diferente
# Na primeira os indices sao as cidades e as temperaturas as linhas
# Na segunda os indices sao os dias de coleta e as cidades sao os titulos das colunas
# Uma com 3 dias de coleta e outra com 4
# IREMOS USAR O COMANDO melt() PARA RECONFIGURAR AS TABELAS.
# Esse comando auxilia para colocarmos o DF de uma maneira que cada linha seja
# uma observacao
# id_vars: sao var que nao serao modificadas e continuarao como colunas
# value_vars: sao as variaveis que passarao a ser "valor" (linhas)

# iniciando o tratamento
# Para o DF1, vamos retirar as cidades como indice e inclui-la como coluna e
# resetar o indice

dtemp1 = dtemp1.rename_axis('city').reset_index()

# apos ajustar a cidade, ainda temos cada dia como uma coluna
# agora vamos criar uma coluna unica para os dias, usando o seu indice
# como o dado que ira indicar o dia da coleta
# para fazer isso aplico o melt, mantendo o city sem alterar

dtemp1_melt = dtemp1.melt(id_vars=['city'])

#Podemos renomear as variaveis (colunas) no DF
tabela1 = dtemp1.melt(id_vars=['city'], value_name='temperature', var_name='day')

# Trabalhando na tabela 2
# Se a tabela 2 for transposta, teremos algo muito similar ao outro DF
# vamos incluir isso no ajuste

dtemp2_tran = dtemp2.transpose().rename_axis('city').reset_index()

# Criando a tabela final com o indice como coluna e renomeando campos

tabela2 = dtemp2_tran.melt(id_vars=['city'], value_name='temperature', var_name='day')

# A coleta da tabela 2 comecou 3 dias apos a primeira. Vamos somar 3 dias nos indices
# vamos pegar o ultimo dia da tabela 1

last_day = tabela1['day'].max()

tabela2['day'] = tabela2['day'].apply(lambda x: x + last_day + 1)

tabela1


,city,day,temperature
0,São Paulo,0,23
1,Fortaleza,0,30
2,Montevideo,0,18
3,London,0,9
4,São Paulo,1,21
5,Fortaleza,1,29
6,Montevideo,1,21
7,London,1,10
8,São Paulo,2,20
9,Fortaleza,2,28


In [2]:
# Formas de combinar os DFs
# merge: combina os dados vinculando linhas por uma ou mais chave.
# Fusao com base em
# 1. Valores de colunas, uma em cada DF
# 2. Valores de uma coluna em um dos DF e indices do outro
# 3. Valores dos indices em ambos os DFs (ou serie)

# concatenate: combina dados a partir de um eixo especifico

# construindo 2 DFs que serao fundidos com o metodo merge
import pandas as pd
df1 = pd.DataFrame({'key': ['a','a','b','b','c','a'],
                    'data1': range(10, 70, 10)})
df2 = pd.DataFrame({'key': ['b','a','a','b'],
                    'data2': range(4)})

# fusao com base nos valores de KEY (como chave)
pd.merge(df1, df2, on='key')
# pega a 1 chave do DF e procura no 2 DF. Ou seja chave que tem valores repetidos, vao gerar linhas repetidas.
# vai mostrar 2 colunas com os resultados possiveis de cada indice
# Apenas os valores que aparecem em ambos sao utilizados no merge


,key,data1,data2
0,a,10,1
1,a,10,2
2,a,20,1
3,a,20,2
4,b,30,0
5,b,30,3
6,b,40,0
7,b,40,3
8,a,60,1
9,a,60,2


In [3]:
# Utilizando o parametro how
# no antetiror fizemos assim: pd.merge(df1, df2, on='key')
import pandas as pd
df1 = pd.DataFrame({'key': ['a','a','b','b','c','a'],
                    'data1': range(10, 70, 10)})
df2 = pd.DataFrame({'key': ['b','a','a','b'],
                    'data2': range(4)})

pd.merge(df1, df2, on='key', how= 'outer')

# Neste caso traz tb o C, que so existe em um DF. Para o inexistente ele preenche com NaN


,key,data1,data2
0,a,10,1.0
1,a,10,2.0
2,a,20,1.0
3,a,20,2.0
4,a,60,1.0
5,a,60,2.0
6,b,30,0.0
7,b,30,3.0
8,b,40,0.0
9,b,40,3.0


In [12]:
# Merge baseado nos valores de coluna em DataFrame e rotudos no outro DataFrame
# Para realizar o merging com base em colunas e rotulos devemos especificar os
# parametros: left_on, right_on, left_index e right_index
# left_on = nome da coluna do DF da esquerda (primeiro parametro)
# left_index = True indica que o indice no DF da esquerda sera utilizado
# right_on = nome da coluna do DF da direita (segundo parametro)
# right_index = True indica que o indice no DF da direita (segundo parametro) sera utilizado

import pandas as pd
dcity = pd.DataFrame({'country':['BR','BR','UR']}, index=[['Sao Paulo','Fortaleza','Montevideo']])
dcity

,country
Sao Paulo,BR
Fortaleza,BR
Montevideo,UR


In [27]:
# merge usando a coluna city da tabela 1 (de temperatura) e os indices em dcity
#
import pandas as ps
pd.merge(tabela1, dcity, left_on='city', right_index=True, how='outer')



,city,day,temperature,country
1,Fortaleza,0,30.0,BR
5,Fortaleza,1,29.0,BR
9,Fortaleza,2,28.0,BR
3,London,0,9.0,NaN
7,London,1,10.0,NaN
11,London,2,13.0,NaN
2,Montevideo,0,18.0,UR
6,Montevideo,1,21.0,UR
10,Montevideo,2,20.0,UR
11,Sao Paulo,NaN,NaN,BR


In [28]:
# Comando contcat - mais simples
# axis = 0 - concatena por linhas
# axis = 1 - concatena por colunas. Quando empregado em um conjunto de series resulta em um DataFrame
import pandas as pd
s1 = pd.Series([0,1], index=['a','b'])
s2 = pd.Series([2,3,4], index=['a','c','d'])
s3 = pd.Series([1,5,6], index=['a','b','e'])

print(s1)
print(s2)
print(s3)


a    0
b    1
dtype: int64
a    2
c    3
d    4
dtype: int64
a    1
b    5
e    6
dtype: int64


In [34]:
import pandas as pd
# se nao definir nada usa o padrao axis = 0
serie1 = pd.concat([s1,s2,s3])
serie1

,0
a,0
b,1
a,2
c,3
d,4
a,1
b,5
e,6


In [33]:
import pandas as pd
serie1 = pd.concat([s1,s2,s3], axis =1)
serie1

,0,1,2
a,0.0,2.0,1.0
b,1.0,NaN,5.0
c,NaN,3.0,NaN
d,NaN,4.0,NaN
e,NaN,NaN,6.0


In [35]:
tabela_temperaturas = pd.concat([tabela1, tabela2], axis=0)
print(tabela_temperaturas)

          city day  temperature
0    São Paulo   0           23
1    Fortaleza   0           30
2   Montevideo   0           18
3       London   0            9
4    São Paulo   1           21
5    Fortaleza   1           29
6   Montevideo   1           21
7       London   1           10
8    São Paulo   2           20
9    Fortaleza   2           28
10  Montevideo   2           20
11      London   2           13
0    São Paulo   3           25
1    Fortaleza   3           35
2   Montevideo   3           20
3       London   3           14
4    São Paulo   4           27
5    Fortaleza   4           32
6   Montevideo   4           18
7       London   4           15
8    São Paulo   5           23
9    Fortaleza   5           31
10  Montevideo   5           21
11      London   5           12
12   São Paulo   6           25
13   Fortaleza   6           29
14  Montevideo   6           23
15      London   6           13
